In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd

## timeseries.py

In [3]:
a = np.random.uniform(size=(1000, 1000))

a.shape

(1000, 1000)

In [4]:
%timeit vbt.timeseries.set_by_mask_nb(a, np.isnan(a), 0)

print(vbt.timeseries.set_by_mask_nb(a, np.isnan(a), 0).shape)

3.65 ms ± 41.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [5]:
%timeit vbt.timeseries.fillna_nb(a, 0)

print(vbt.timeseries.fillna_nb(a, 0).shape)

3.72 ms ± 34.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [6]:
%timeit vbt.timeseries.prepend_nb(a, 1000, np.nan)

print(vbt.timeseries.prepend_nb(a, 1000, np.nan).shape)

5.03 ms ± 47.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(2000, 1000)


In [7]:
%timeit vbt.timeseries.fshift_nb(a, 100)

print(vbt.timeseries.fshift_nb(a, 100).shape)

3.8 ms ± 35.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [8]:
%timeit vbt.timeseries.diff_nb(a, 100)

print(vbt.timeseries.diff_nb(a, 100).shape)

26.3 ms ± 239 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


In [9]:
%timeit vbt.timeseries.pct_change_nb(a)

print(vbt.timeseries.pct_change_nb(a).shape)

11.3 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [10]:
%timeit vbt.timeseries.ffill_nb(a)

print(vbt.timeseries.ffill_nb(a).shape)

9.48 ms ± 519 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [11]:
%timeit vbt.timeseries.cumsum_nb(a)

print(vbt.timeseries.cumsum_nb(a).shape)

12.8 ms ± 2.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


In [12]:
%timeit vbt.timeseries.cumprod_nb(a)

print(vbt.timeseries.cumprod_nb(a).shape)

11.9 ms ± 493 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [13]:
%timeit vbt.timeseries.rolling_mean_nb(a, 10)

print(vbt.timeseries.rolling_mean_nb(a, 10).shape)

13.4 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [14]:
%timeit vbt.timeseries.rolling_std_nb(a, 10)

print(vbt.timeseries.rolling_std_nb(a, 10).shape)

17.5 ms ± 82.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [15]:
%timeit vbt.timeseries.expanding_max_nb(a)

print(vbt.timeseries.expanding_max_nb(a).shape)

9.26 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [16]:
%timeit vbt.timeseries.ewm_mean_nb(a, 10)

print(vbt.timeseries.ewm_mean_nb(a, 10).shape)

12.2 ms ± 65.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [17]:
%timeit vbt.timeseries.ewm_std_nb(a, 10)

print(vbt.timeseries.ewm_std_nb(a, 10).shape)

21.1 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


## signals.py

In [18]:
%timeit vbt.signals.generate_random_entries_nb((1000, 1000), 100, 2, 42)

entries = vbt.signals.generate_random_entries_nb((1000, 1000), 100, 2, 42)
print(entries.shape)

2.74 ms ± 13.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [19]:
%timeit vbt.signals.generate_random_exits_nb(entries, 42)

exits = vbt.signals.generate_random_exits_nb(entries, 42)
print(exits.shape)

28.4 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


In [22]:
from numba import njit, b1

@njit
def exit_func_nb(entries, col_idx, prev_idx, next_idx):
    # First value after entry
    entries = entries[:, col_idx]
    a = np.full(entries.shape, False, dtype=b1)
    a[prev_idx+1:] = True
    return a

In [23]:
%timeit vbt.signals.generate_exits_nb(entries, exit_func_nb, True)

print(vbt.signals.generate_exits_nb(entries, exit_func_nb, True).shape)
# Iterative functions are the most expensive

556 ms ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [24]:
@njit
def entry_func_nb(exits, col_idx, prev_idx, next_idx, a):
    # First value after last entry
    exits = exits[:, col_idx]
    a = a[:, col_idx]
    entries = np.full(exits.shape, False, dtype=b1)
    entries[a > 0.5] = True
    return entries

@njit
def exit_func_nb(entries, col_idx, prev_idx, next_idx, a):
    # First value after last exit
    entries = entries[:, col_idx]
    a = a[:, col_idx]
    exits = np.full(entries.shape, False, dtype=b1)
    exits[a < 0.5] = True
    return exits

In [25]:
%timeit vbt.signals.generate_entries_and_exits_nb(a.shape, entry_func_nb, exit_func_nb, a)

print(vbt.signals.generate_entries_and_exits_nb(a.shape, entry_func_nb, exit_func_nb, a)[0].shape)
# Iterative functions are the most expensive

3.88 s ± 70.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [26]:
%timeit vbt.signals.rank_true_nb(entries, True)

print(vbt.signals.rank_true_nb(entries, True).shape)

3.85 ms ± 413 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [27]:
%timeit vbt.signals.rank_false_nb(entries, True)

print(vbt.signals.rank_false_nb(entries, True).shape)

6.23 ms ± 992 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [28]:
%timeit vbt.signals.shuffle(entries, 42)

print(vbt.signals.shuffle(entries, 42).shape)

15.7 ms ± 74.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [29]:
%timeit vbt.signals.avg_distance_nb(entries)

print(vbt.signals.avg_distance_nb(entries).shape)

3.87 ms ± 52.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000,)


In [30]:
%timeit vbt.signals.prepend_nb(entries, 100, False)

print(vbt.signals.prepend_nb(entries, 100, False).shape)

1.4 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
(1100, 1000)


In [31]:
%timeit vbt.signals.fshift_nb(entries, 100)

print(vbt.signals.fshift_nb(entries, 100).shape)

1.34 ms ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
(1000, 1000)


In [32]:
%timeit vbt.signals.first_true_nb(entries, True)

print(vbt.signals.first_true_nb(entries, True).shape)

4.53 ms ± 75.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [33]:
%timeit vbt.signals.first_false_nb(entries, True)

print(vbt.signals.first_false_nb(entries, True).shape)

6.26 ms ± 44.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [34]:
%timeit vbt.signals.stoploss_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True)

print(vbt.signals.stoploss_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True).shape)

1.7 ms ± 31.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000,)


In [35]:
%timeit vbt.signals.stoploss_exits_nb(entries, a, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True)

print(vbt.signals.stoploss_exits_nb(entries, a, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True).shape)

803 ms ± 5.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [36]:
stops = np.arange(0, 1, 0.01)
stops = np.tile(stops[:, None][:, None], (1, a.shape[0], a.shape[1]))

print(stops.shape)

(100, 1000, 1000)


In [37]:
%timeit vbt.signals.stoploss_exits_nb(entries, a, stops, True, True)

print(vbt.signals.stoploss_exits_nb(entries, a, stops, True, True).shape)

1min 7s ± 3.02 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [39]:
%timeit vbt.signals.trailstop_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True)

print(vbt.signals.trailstop_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True).shape)

2.18 ms ± 392 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000,)


In [40]:
%timeit vbt.signals.trailstop_exits_nb(entries, a, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True)

print(vbt.signals.trailstop_exits_nb(entries, a, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True).shape)

649 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [41]:
%timeit vbt.signals.trailstop_exits_nb(entries, a, stops, True, True)

print(vbt.signals.trailstop_exits_nb(entries, a, stops, True, True).shape)

1min 1s ± 1.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


## positions.py

In [42]:
%timeit vbt.positions.from_signals_nb(entries, exits)

positions = vbt.positions.from_signals_nb(entries, exits)
print(positions.shape)

5.07 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


## portfolio.py

In [43]:
%timeit vbt.portfolio.equity_nb(a, positions, 1000, np.full_like(a, 0.1), np.full_like(a, 0.1))

equity = vbt.portfolio.equity_nb(a, positions, 1000, np.full_like(a, 0.1), np.full_like(a, 0.1))
print(equity.shape)

17.4 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [44]:
%timeit vbt.portfolio.trade_profits_nb(equity, positions)

print(vbt.portfolio.trade_profits_nb(equity, positions).shape)

7.69 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [45]:
%timeit vbt.portfolio.trade_returns_nb(equity, positions)

print(vbt.portfolio.trade_returns_nb(equity, positions).shape)

7.69 ms ± 69.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


## strategy.py

In [46]:
%timeit vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, False, True)

print(vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, False, True)[0].shape)

2.6 s ± 44.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [47]:
%timeit vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, True, True)

print(vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, True, True)[0].shape)
# ewm is much more efficient

2.48 s ± 5.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [48]:
# Test caching
%timeit vbt.strategy.dmac_nb(a, np.full(100, 2), np.full(100, 3), True, True)

print(vbt.strategy.dmac_nb(a, np.full(100, 2), np.full(100, 3), True, True)[0].shape)

1.02 s ± 8.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [49]:
%timeit vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), False, True)

print(vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), False, True)[0].shape)

5.64 s ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [50]:
%timeit vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), True, True)

print(vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), True, True)[0].shape)

5.87 s ± 33.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [51]:
# Test caching
%timeit vbt.strategy.bb_nb(a, np.full(100, 2), np.full(100, 1), True, True)

print(vbt.strategy.bb_nb(a, np.full(100, 2), np.full(100, 1), True, True)[0].shape)

2.12 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [52]:
%timeit vbt.strategy.rsi_nb(a, np.arange(1, 100), False, True)

print(vbt.strategy.rsi_nb(a, np.arange(1, 100), False, True).shape)

4.66 s ± 32.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [53]:
%timeit vbt.strategy.rsi_nb(a, np.arange(1, 100), True, True)

print(vbt.strategy.rsi_nb(a, np.arange(1, 100), True, True).shape)

4.43 s ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [54]:
# Test caching
%timeit vbt.strategy.rsi_nb(a, np.full(100, 2), True, True)

print(vbt.strategy.rsi_nb(a, np.full(100, 2), True, True).shape)

910 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)
